In [1]:
import cv2
import dlib
import numpy as np
import pyautogui as pag

%run Config.ipynb
%run Utils.ipynb

In [ ]:
cap=cv2.VideoCapture(0)

while True:
    _, frame=cap.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects=detector(gray, 1) # detects the presence of face in the given frame
    
    for (i, rect) in enumerate(rects):
        
        shape=predictor(gray, rect) #predicts the facial landmarks in the detected face
                
        shape=npshape(shape) #converts the shape into a numpy array of 68x2 where each row corresponds 
                             #to the coordinates of the landmarks in the image
            
#         for (x, y) in shape:
#             cv2.circle(frame, (x,y), 2, (255,0,0), -1)
    
        mask=np.zeros(frame.shape[:2], dtype=np.uint8) #declares an matrix of same size as frame
        
        mask=mask_of_eye(mask, left, shape) #fills the area covered by the eyes with white using fillConvexPoly method
        mask=mask_of_eye(mask, right, shape)
        
        mask=cv2.dilate(mask, kernel, 5) #a kernel of 9x9 is used to convolve over the image
        
        eyes=cv2.bitwise_and(frame, frame, mask=mask) #bitiwise and operation makes the entire frame black 
                                                      #except where the eyes are coloured. 
        
        #makes the background white
        mask=(eyes==[0,0,0]).all(axis=2)
        eyes[mask]=[255,255,255]
        
        gray_eyes=cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
        
        
        #thresholding of frame is done for contouring
        _,thresh=cv2.threshold(gray_eyes, 100, 255, cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=4)
        thresh = cv2.medianBlur(thresh, 3)
        
        mid = (shape[42][0] + shape[39][0]) // 2
        
        thresh = cv2.bitwise_not(thresh) #Cotouring requires that the object being 
                                         #detected should be white with a black background.
        
        #contours are calculated for left and right eyes. Moments are used to calculate centroid of the eyes.
        cx1, cy1=contouring(thresh[:, 0:mid], mid, frame)
        cx2, cy2=contouring(thresh[:, mid:], mid, frame, True)
        

        #The ratio of width and height of the eyes with that of screen resolution is calculated. 
        #The facial landmarks used for this are 40, 37, 39, 41
        
        ratio_x=2000/abs(shape[39][0]-shape[36][0]) 
        ratio_y=800/abs(shape[38][1]-shape[40][1])
        
        if prev1==None and prev2==None:
            if cx1==None or cx2==None:
                prev1=(0,0)
                prev2=(0,0)
            else:
                prev1=(cx1, cy1)
                prev2=(cx2, cy2)
            
        if cx1==None or cy1==None:
            cx1, cy1=prev1
            
        if cx2==None or cy2==None:
            cx2, cy2=prev2
        
        #Difference between the previous and current position of the eyes is used 
        #to calculate the shift in position of cursor
        
        horz=(prev1[0]-cx1+prev2[0]-cx2)//2
        vert=(cy1-prev1[1]+cy2-prev2[1])//2
        
        pag.moveRel(horz*ratio_x,vert*ratio_y) #move horizontally and vertically
            
            
        prev1=(cx1, cy1)
        prev2=(cx2, cy2)

        frame=cv2.flip(frame, 1)
        cv2.imshow('res', frame)        
        
    if cv2.waitKey(1) & 0xff==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break